<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/space_travel_demo_MISTRAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env --upgrade -q
!pip install mistralai --upgrade

In [2]:
import os
import colab_env

from mistralai import Mistral
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

api_key = os.environ.get("MISTRAL_API_KEY")

if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()


model_list = client.models.list()

# Print all model IDs
for model_info in model_list.data:
    print(model_info.id)

model = "magistral-medium-latest"

messages = [{"role": "user", "content": "What is the best Cuban poet?"}]

chat_response = client.chat.complete(
    model=model,  # Pass the model ID string
    messages=messages,
)
print('\n')
print(chat_response.choices[0].message.content)

Mounted at /content/gdrive
mistral-medium-2505
mistral-medium-latest
mistral-medium
ministral-3b-2410
ministral-3b-latest
ministral-8b-2410
ministral-8b-latest
open-mistral-7b
mistral-tiny
mistral-tiny-2312
open-mistral-nemo
open-mistral-nemo-2407
mistral-tiny-2407
mistral-tiny-latest
open-mixtral-8x7b
mistral-small
mistral-small-2312
open-mixtral-8x22b
open-mixtral-8x22b-2404
mistral-small-2409
mistral-large-2407
mistral-large-2411
mistral-large-latest
pixtral-large-2411
pixtral-large-latest
mistral-large-pixtral-2411
codestral-2501
codestral-latest
codestral-2412
codestral-2411-rc5
devstral-small-2505
devstral-small-latest
pixtral-12b-2409
pixtral-12b
pixtral-12b-latest
mistral-small-2501
mistral-small-2503
mistral-small-2506
mistral-small-latest
mistral-saba-2502
mistral-saba-latest
magistral-medium-2506
magistral-medium-latest
magistral-small-2506
magistral-small-latest
mistral-embed
codestral-embed
codestral-embed-2505
mistral-moderation-2411
mistral-moderation-latest
mistral-ocr-

## EARTH TO MARS

In [3]:
import os
from typing import List, Dict
from datetime import datetime, timedelta
from mistralai import Mistral # Corrected: Using the import exactly as in your reference


# Assume a global variable for the Mistral client and model name
mistral_client = None
mistral_model_name = None

def load_mistral_model_client():
    """Loads the Mistral client and sets the model name according to the provided reference."""
    global mistral_client, mistral_model_name

    if mistral_client is None:
        api_key = os.environ.get("MISTRAL_API_KEY")
        if not api_key:
            print("Error: MISTRAL_API_KEY environment variable not set.")
            print("Please set your Mistral API key before running this script.")
            exit()

        mistral_client = Mistral(api_key=api_key) # Initialize client exactly as in your reference
        print("Mistral client loaded successfully.")

    if mistral_model_name is None:
        mistral_model_name = "magistral-medium-latest" # Directly use the specified model name
        print(f"Mistral model set to: {mistral_model_name}")

def generate_flight_plan_with_llm(prompt: str, system_prompt: str = None, model: str = "mistral", max_tokens: int = 1024, temperature: float = 0.7):
    """Generates a response from the Mistral LLM."""
    load_mistral_model_client()

    messages = []
    if system_prompt:
        # Note: Without ChatMessage, roles must be directly provided in dicts.
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})

    try:
        chat_response = mistral_client.chat.complete(
            model=mistral_model_name,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        response_text = chat_response.choices[0].message.content
        return response_text.strip()
    except Exception as e:
        print(f"Error generating response from Mistral LLM: {e}")
        return f"Error: Could not generate response. {e}"


def generate_flight_plan_iteratively(prompt: str, system_prompt: str = None, model: str = "mistral", max_tokens_per_iteration: int = 512, temperature: float = 0.7):
    """Generates a flight plan iteratively from the Mistral LLM."""
    load_mistral_model_client()
    full_response = ""
    current_prompt_text = prompt
    messages_history = []

    if system_prompt:
        messages_history.append({"role": "system", "content": system_prompt})

    while True:
        current_messages = messages_history + [{"role": "user", "content": current_prompt_text}]
        try:
            chat_response = mistral_client.chat.complete(
                model=mistral_model_name,
                messages=current_messages,
                temperature=temperature,
                max_tokens=max_tokens_per_iteration,
            )
            chunk = chat_response.choices[0].message.content
            full_response += chunk
            messages_history.append({"role": "assistant", "content": chunk})

            if len(chunk.strip()) < 20 and not full_response.endswith("Continue:"):
                break
            elif chat_response.choices[0].finish_reason == "stop":
                break

            current_prompt_text = f"{chunk} Continue:"
            messages_history.append({"role": "user", "content": "Continue:"})

        except Exception as e:
            print(f"Error generating iterative response from Mistral LLM: {e}")
            full_response += f"\nError during iterative generation: {e}"
            break

    return full_response.strip()


class SpaceFlightPlanningAgent:
    def __init__(self, llm_models: List[str]):
        self.llm_models = llm_models

    def generate_flight_plan(
        self,
        origin: str,
        destination: str,
        spacecraft: str,
        mission_objectives: List[str],
        constraints: List[str],
    ) -> Dict[str, Dict[str, str]]:
        system_prompt = f"""
You are a helpful and informative space flight planner.
You are tasked with generating a detailed flight plan for a mission from {origin} to {destination}
using a {spacecraft}. The mission objectives are: {mission_objectives}. The flight plan
must satisfy the following constraints: {constraints}.
Consider these factors:
Orbital mechanics (including Hohmann transfer orbits, delta-v calculations, and
apogee/perigee for Mars; translunar injection, lunar orbit insertion, and descent/ascent for Moon)
Spacecraft capabilities (delta-v, thrust-to-weight ratio, specific impulse)
Gravitational forces of relevant celestial bodies (Sun, Earth, Mars, Moon, etc.)
Radiation exposure (minimize exposure by considering solar activity and trajectory)
Communication windows (ensure contact with Earth throughout the mission)
Launch opportunities (identify optimal launch windows based on planetary/lunar alignment)
"""
        flight_plan_sections = {}

        if destination == "Mars":
            flight_plan_sections = {
                "Launch Date and Time": """Provide a **specific** launch date and time in UTC, consider
Do not provide a range of dates or suggest future windows. Choose a date
starting from 2025-07-03 20:27:07 UTC.
""",
                "Trajectory": {
                    "initial_prompt": "Provide a summary of the key phases and maneuvers of the trajectory.",
                    "follow_up_prompts": [
                        "Describe the launch process, including the type of launch vehicle, the target orbit, and initial burn.",
                        "Explain the Hohmann transfer maneuver, including the delta-v requirements, the timing, and the resulting orbit parameters.",
                        "Detail the interplanetary cruise phase, including the duration, the communication strategy, and any mid-course corrections.",
                        "Describe the approach to Mars and the Mars orbit insertion maneuver, including the braking burn and the final orbital parameters."
                    ]
                },
                "Maneuver Schedule": "Detail each maneuver with its purpose, delta-v requirement, and timing.",
                "Communication Plan": "Explain how communication with Earth will be maintained throughout the mission, including DSN usage and data transmission rates.",
                "Contingency Plans": "Describe contingency plans for various scenarios, such as launch aborts, engine failures, communication loss, or medical emergencies."
            }
        elif destination == "Moon":
            flight_plan_sections = {
                "Launch Date and Time": f"""Provide a **specific** launch date and time in UTC, consider
Do not provide a range of dates or suggest future windows. Choose a date
starting from {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}.
""",
                "Trajectory": {
                    "Launch": "Describe the launch process, including the launch vehicle, launch site, and initial orbit.",
                    "Earth Departure and Translunar Injection": "Describe the Earth departure and translunar injection (TLI) maneuver, including delta-v and trajectory.",
                    "Lunar Orbit Insertion": "Describe the lunar orbit insertion maneuver, including the braking burn and the target lunar orbit.",
                    "Lunar Descent and Landing": "Describe the lunar descent and landing process, including key phases, guidance systems, and landing site considerations.",
                    "Lunar Surface Operations": "Describe the planned activities on the lunar surface, including duration, scientific experiments, and mobility.",
                    "Lunar Ascent and Return": "Describe the lunar ascent and return trajectory, including lunar orbit rendezvous (if applicable) and trans-Earth injection (TEI).",
                    "Earth Re-entry and Landing": "Describe the Earth re-entry and landing process, including re-entry corridor, heat shield, and landing site."
                },
                "Maneuver Schedule": "Detail each maneuver with its purpose, delta-v requirement, and timing.",
                "Communication Plan": "Explain how communication with Earth will be maintained throughout the mission.",
                "Contingency Plans": "Describe contingency plans for various scenarios, such as launch aborts, engine failures, communication loss, or medical emergencies."
            }

        all_flight_plans = {}

        for model_key in self.llm_models:
            if model_key != "mistral":
                print(f"Warning: Skipping unsupported model '{model_key}'. Only 'mistral' is supported in this migration.")
                continue

            flight_plan = {}
            temperature = 0.7

            for section_name, section_instructions in flight_plan_sections.items():
                if section_name == "Trajectory":
                    if destination == "Mars":
                        if isinstance(section_instructions, dict) and "initial_prompt" in section_instructions:
                            prompt = section_instructions["initial_prompt"]
                            trajectory_details = [generate_flight_plan_with_llm(prompt, system_prompt, 'mistral', temperature=temperature)]
                            for sub_question in section_instructions["follow_up_prompts"]:
                                prompt = f"""
Specifically, focus on the {section_name}:
{sub_question}
"""
                                trajectory_details.append(generate_flight_plan_with_llm(prompt, system_prompt, 'mistral', temperature=temperature))
                            flight_plan[section_name] = "\n".join(trajectory_details)
                    elif destination == "Moon":
                        trajectory_details = ""
                        for subsection_name, subsection_instructions in section_instructions.items():
                            prompt = f"""
Specifically, focus on the {subsection_name} of the Trajectory:
{subsection_instructions}
"""
                            response = generate_flight_plan_iteratively(prompt, system_prompt, "mistral", temperature=temperature)
                            trajectory_details += f"\n**{subsection_name}:**\n{response}\n"
                        flight_plan[section_name] = trajectory_details
                else:
                    prompt = f"""
Specifically, focus on the {section_name}:
{section_instructions}
"""
                    response = generate_flight_plan_with_llm(prompt, system_prompt, "mistral", temperature=temperature)
                    retry_count = 0
                    temp = temperature
                    while len(response) < 100 and retry_count < 3:
                        print(f"Response for {section_name} was truncated or incomplete. Retrying with increased temperature...")
                        response = generate_flight_plan_with_llm(prompt, system_prompt, "mistral", temp)
                        retry_count += 1
                        temp += 0.1
                    if retry_count >= 3:
                        break
                    flight_plan[section_name] = response
            all_flight_plans[model_key] = flight_plan
        return all_flight_plans

if __name__ == "__main__":
    # Ensure your MISTRAL_API_KEY environment variable is set.

    agent = SpaceFlightPlanningAgent(["mistral"])

    #--- Earth to Mars Mission (Mistral LLM)
    print("-- Earth to Mars Mission (Mistral LLM) --")
    print("Starting Mars flight plan generation...")
    print('\n')
    all_flight_plans_mars = agent.generate_flight_plan(
        origin="Earth",
        destination="Mars",
        spacecraft="Orion spacecraft",
        mission_objectives=["Land on Mars", "Collect soil samples"],
        constraints=["Minimize radiation exposure", "Ensure crew safety"]
    )
    print("Mars flight plan generation complete.")
    print('\n')
    for model, flight_plan in all_flight_plans_mars.items():
        print(f"Flight plan generated by {model}:")
        for section_name, section_content in flight_plan.items():
            print(f"{section_name}:\n{section_content}\n")



-- Earth to Mars Mission (Mistral LLM) --
Starting Mars flight plan generation...


Mistral client loaded successfully.
Mistral model set to: magistral-medium-latest
Mars flight plan generation complete.


Flight plan generated by mistral:
Launch Date and Time:
<think>
Alright, I have a task to generate a detailed flight plan for a mission from Earth to Mars using an Orion spacecraft. The mission objectives are to land on Mars and collect soil samples. The constraints are to minimize radiation exposure and ensure crew safety.

First, I need to consider the orbital mechanics. A Hohmann transfer orbit is typically used for missions to Mars because it is the most fuel-efficient trajectory. This involves two main engine burns: one to leave Earth's orbit and another to enter Mars' orbit.

But before diving into the specifics of the Hohmann transfer, I need to determine a suitable launch date. The launch window to Mars is crucial because it depends on the relative positions of Earth and Mars

## Earth to Moon

In [ ]:
if __name__ == "__main__":

    agent = SpaceFlightPlanningAgent(["mistral"])

     #--- Earth to Moon Mission (Mistral LLM)
    print("\n--- Earth to Moon Mission (Mistral LLM) --")
    print("Starting Moon flight plan generation...")
    all_flight_plans_moon = agent.generate_flight_plan(
        origin="Earth",
        destination="Moon",
        spacecraft="Orion spacecraft",
        mission_objectives=["Land on the Moon", "Conduct scientific experiments"],
        constraints=["Minimize radiation exposure", "Ensure crew safety"]
    )
    print("\n--- Moon flight plan generation complete. --")
    print('\n')
    for model, flight_plan in all_flight_plans_moon.items():
        print(f"Flight plan generated by {model}:")
        print('\n')
        for section_name, section_content in flight_plan.items():
            print(f"{section_name}:\n{section_content}\n")


--- Earth to Moon Mission (Mistral LLM) --
Starting Moon flight plan generation...
